In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from datascience import *
import numpy as np
import random
from sodapy import Socrata

In [18]:
client = Socrata("data.lacity.org", None)

In [19]:
#la24_results = client.get("xjgu-z4ju")
all_results = []
batch_size = 1000
offset = 0

while True:
    # Fetch a batch of rows
    batch = client.get("xjgu-z4ju", limit=batch_size, offset=offset)
    if not batch:  # Stop if no more rows are returned
        break
    all_results.extend(batch)
    offset += batch_size  # Increment offset for the next batch

In [ ]:
la24_results_df = pd.DataFrame.from_records(all_results)

la24_results_df.describe()

In [21]:
la24_results_table = Table().from_df(la24_results_df)

In [37]:
areas = la24_results_table.group("area_occ").column(0)

counts = la24_results_table.group("area_occ").column(1)

counts = counts/100

colors = [f"#{random.randint(0, 0xFFFFFF):06x}" for _ in areas]

la24_results_latitudes = [33.9458, 34.0443, 34.2635, 34.2723, 33.7734, 
 34.0498, 34.1016, 34.2728, 34.187, 34.0224, 
 34.1222, 34.0536, None, 33.9935, 34.0622, 
 33.9461, 34.0101, 34.2154, 34.1867, 34.0452, 
 34.187, 34.0622]

la24_results_longitudes = [-118.2917, -118.2528, -118.5075, -118.4148, -118.262, 
 -118.212, -118.3296, -118.4482, -118.3861, -118.2448, 
 -118.2096, -118.277, None, -118.4561, -118.2766, 
 -118.2397, -118.3091, -118.597, -118.4493, -118.4426, 
 -118.5379, -118.308]

la24_data_table = Table().with_columns("Areas", areas, "Latitude", la24_results_latitudes, "Longitude", la24_results_longitudes, "colors", colors, "areas", counts, "labels", [areas + ': ' + str(counts) + ' (hundreds)' for areas, counts in zip(areas, counts)])

In [38]:
Circle.map_table(
    la24_data_table.where("Latitude", are.not_equal_to(None))
                  .select('Latitude', 'Longitude', 'Areas', 'colors', 'areas', 'labels'),
    clustered_marker=False
)

In [55]:
la24_area_call = la24_results_table.group([1,6]).sort('count', descending=True)
la24_area_call

area_occ,call_type_text,count
Outside,CODE 6,333199
Outside,902 TRAFFIC STOP,35101
Central,CODE 6,31472
Newton,CODE 6,22731
Southwest,CODE 6,19745
Pacific,CODE 6,18745
77th Street,CODE 6,15140
Southeast,CODE 6,14839
Mission,CODE 6,14440
N Hollywood,CODE 6,13072


AttributeError: 'NoneType' object has no attribute 'show'